<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Откройте-и-изучите-файл" data-toc-modified-id="Откройте-и-изучите-файл-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Откройте и изучите файл</a></span></li><li><span><a href="#Разбейте-данные-на-выборки" data-toc-modified-id="Разбейте-данные-на-выборки-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Разбейте данные на выборки</a></span></li><li><span><a href="#Исследуйте-модели" data-toc-modified-id="Исследуйте-модели-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Исследуйте модели</a></span><ul class="toc-item"><li><span><a href="#Дерево-решений" data-toc-modified-id="Дерево-решений-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Дерево решений</a></span></li><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Случайный лес</a></span></li><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li></ul></li><li><span><a href="#Проверьте-модель-на-тестовой-выборке" data-toc-modified-id="Проверьте-модель-на-тестовой-выборке-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверьте модель на тестовой выборке</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод</a></span></li><li><span><a href="#(бонус)-Проверьте-модели-на-адекватность" data-toc-modified-id="(бонус)-Проверьте-модели-на-адекватность-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>(бонус) Проверьте модели на адекватность</a></span></li></ul></div>

# Рекомендация тарифов

В нашем распоряжении данные о поведении клиентов, которые уже перешли на новые тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф.

## Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from joblib import dump
import numpy as np
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score


In [2]:
try:
     df = pd.read_csv(r"C:\Проекты Яндекс.Практикум\6. Введение в машинное обучение. Мобильная связь\Датасет\users_behavior.csv")
except: 
    df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Импортировали библиотеки

Загрузили и открыли файл с данными

Датасет состоит из 3214 строк и 5 столбцов. Есть данные:

calls - количество звонков за месяц;
minutes — суммарная длительность звонков в минутах за месяц,
messages — количество sms-сообщений за месяц,
mb_used — израсходованный интернет-трафик в Мб за месяц,
is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

Все столбцы кроме "is_ultra" имеют тип данных float. "is_ultra" типа данных - int. Пропусков нет


In [5]:
df.duplicated().sum()

0

Не найдено явных дубликатов


In [6]:
df['is_ultra'].unique()

array([0, 1])

Убедились, что столбец is_ultra заполнен только значениями тарифов 0 и 1

In [7]:
df['calls'].describe()

count    3214.000000
mean       63.038892
std        33.236368
min         0.000000
25%        40.000000
50%        62.000000
75%        82.000000
max       244.000000
Name: calls, dtype: float64

Изучили столбец с количеством звонков. Медианное и среднее практически совпадают 62 и 63 звонка в месяц, т. е. выбросов нет. Максимальное количество звонков 244, минимальное 0 (звонки не совершались)

In [8]:
df['minutes'].describe()

count    3214.000000
mean      438.208787
std       234.569872
min         0.000000
25%       274.575000
50%       430.600000
75%       571.927500
max      1632.060000
Name: minutes, dtype: float64

Изучили столбец с общей продолжительностью звонков за месяц. Медиана и средняя также близки: 430 и 438 (около 7 часов). Максимальная общая продолжительность за месяц составила 1632 минуты, т. е. 27 часов. То есть из 30 суток некий абонент проговорил в общей сложности сутки. 

In [9]:
df['messages'].describe()

count    3214.000000
mean       38.281269
std        36.148326
min         0.000000
25%         9.000000
50%        30.000000
75%        57.000000
max       224.000000
Name: messages, dtype: float64

Изучили столбец с количеством отправленных за месяц сообщений. В среднем абоненты отправляют 30-38 сообщений в месяц. Немного, так как в основном люди пользуются мессенджерами вроде whatsapp и whatsapp и telegram.

In [10]:
df['mb_used'].describe()

count     3214.000000
mean     17207.673836
std       7570.968246
min          0.000000
25%      12491.902500
50%      16943.235000
75%      21424.700000
max      49745.730000
Name: mb_used, dtype: float64

Изучили стобец с количеством потраченных Мб. Медианное значение 16943 Мб, в 2,9 меньше, чем максимальное 49745 Мб. Возможно, пользователи, которые используют почти 50 Гб не имеют возможности пользоваться wifi

**Вывод** 

Датасет заполнен корректно: нет явных дубликатов, пропусков, типы данных не требуют изменений. Всего для анализа 3214 записей. Информация представлена по основным характеристикам тарифов, на которые пользователи опираются при выборе. 

## Разбейте данные на выборки

In [11]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']  

features_train, features_other, target_train, target_other = train_test_split(
    features, target, test_size=0.4, random_state=12345)

features_valid, features_test, target_valid, target_test =\
train_test_split(features_other, target_other, test_size=0.5, random_state=12345)

In [12]:
# Код ревьюера
print(features_train.shape, features_valid.shape, features_test.shape)

(1928, 4) (643, 4) (643, 4)


Разделили данные на 3 выборки: обучающую (60%), валидационную (20%) и тестовую (20%). Целевым признаком выбран солбец is_ultra, обозначающий название тарифа "Смарт" или "Ультра"




## Исследуйте модели

### Дерево решений

In [13]:
for depth in range(1, 6):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)

    model.fit(features_train , target_train)

    predictions_valid = model.predict(features_valid )

    print("max_depth =", depth, ": ", end='')
    print(accuracy_score(target_valid, predictions_valid))

max_depth = 1 : 0.7542768273716952
max_depth = 2 : 0.7822706065318819
max_depth = 3 : 0.7853810264385692
max_depth = 4 : 0.7791601866251944
max_depth = 5 : 0.7791601866251944


Применили модель "Дерево решений". С помощью цикла перебрали значения максимальной глубины от 1 до 5. Для каждого значения максимальной глуибины вывели значение accuracy (доля правильных ответов). Самый высокий показатель accuracy = 0,785 удалось достичь при глубине = 3

### Случайный лес

In [21]:
best_model = None
best_result = 0
for est in range(1, 8):
    model = RandomForestClassifier(random_state=12345, n_estimators=est) 
    model.fit(features_train, target_train) 
    result = model.score(features_valid, target_valid)
    if result > best_result:
        best_model = model
        best_result =  result 
        best_est = est

print("Accuracy наилучшей модели на валидационной выборке:", best_result, 'n_estimators:',  best_est)

Accuracy наилучшей модели на валидационной выборке: 0.7807153965785381 n_estimators: 6


Применили модель "Случайный лес". В цикле перебрали значения n_estimators (количество деревьев) от 1 до 8. Лучший показатель accuracy = 0,782 удалось достичь при количестве деревьев = 8. 

### Логистическая регрессия

In [15]:
model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000) 
model.fit(features_train , target_train) 
model.predict(features_train ) 


dump(model, 'model_9_1.joblib')
print(model.score(features_valid, target_valid)  )

0.7107309486780715


Применили модель "Логическая регрессия". Выбрали алгоритм lbfgs для построения модели, как самый распространенный. Максимальное количество итераций сделали равным 1000. В результате получили долю правильных ответов = 0,71

## Проверьте модель на тестовой выборке

Наиболее эффективной моделью в рамках данного задания считаем "Случайный лес". У нее высокий показатель доли правильных ответов = 0,782. Это немного меньше, чем у Решающего дерева, но мы знаем про склонность дерева к переобучению. Accuracy случайного леса выше, чем у Логической регрессии. 

Мы знаем, что при большом количестве деревьев Лес медленнее работает. В нашей задачи высоких показателей accuracy удалось достичь при 8 деревьях, поэтому считаем, что в данном случае медленная работа нам не грозит


In [16]:
model = RandomForestClassifier(random_state=12345, n_estimators=8) 
model.fit(features_train, target_train) 
result = model.score(features_test, target_test)

predictions_test = model.predict(features_test)
print("Accuracy модели на тестовой выборке:", result)



Accuracy модели на тестовой выборке: 0.7869362363919129


In [17]:
def error_count(targets, predictions):
    targets = targets.tolist()
    predictions = predictions.tolist()
    err = 0
    for i in range(len(targets)):
        if targets[i] != predictions[i]:
            err+=1
    return err

print("Ошибок:", error_count(target_test, predictions_test))

Ошибок: 137


Проверили модель Случаный лес с 8 деревьями на тестовой выборке. Показатель accuracy по прежнему высокий = 0,787. Количество ошибок 137. 

## Вывод

В результате исследования был изучен датасет с данными о поведение пользователей мобильной связи. Данные представлены корректно, без явных дубликатов и пропусков. 

Предоставлена информация по затратам в месяц: количество звонков, общая продолжительность звонков, количество сообщений, объем истраченного интернета, используемый тариф. 

Данные были разделены на 3 выборки: обучающую (60%), валидационную (20%) и тестовую (20%). Целевым признаком выбрано название тарифа "Смарт" или "Ультра"

Применены три модели машинного обучения: Дерево решений (добились доли правильных ответов (accuracy) = 0,785), Случайный лес (accuracy=0,782), Логистическая регрессия (accuracy=0,71).

Для дальнейшей работы и проверки на тестовой выборке взяли модель Случайный лес со значением гиперпараметра "количество деревьев" = 8. Отказались от модели Дерево решений из-за ее склонности к переобучению. Отказались от логистической регресси по причине более низкого значения accuracy. 

На тестовой выборке получили высокое значение accuracy = 0,787. 




## (бонус) Проверьте модели на адекватность

Если я правильно понимаю метрики полноты и точности работают только с двумя вариантами ответов (бинарная классификация). И функции ниже посчитались, так как тарифы в датасете равны 1 или 0.

1 = тариф Ультра, 0 = тариф Смарт

Полагаю, что 1 по умолчанию является значением Positive

Если бы в столбце с тарифом значение имело бы тип object: 'ultra'  и 'smart', то нужно было бы как-то предварительно обозначить, что считать Positive

In [18]:
precision = precision_score(target_test, predictions_test)
precision

0.7115384615384616

Для проверки модели на точность взяли тестовый набор данных. Применили функцию для расчета точности. Получили 0,71. 

71% ответов, обозначенных, как тариф Ультра, действительно, являются тарифом Ультра

In [19]:
recall = recall_score(target_test, predictions_test)
recall

0.5467980295566502

Для проверки модели на полноту также взяли тестовый набор данных. Полнота = 0,55. 

Из всех значений Ультра модель только 55% предсказала правильно

**Похоже модель старается как можно чаще предсказать тариф Ультра, но попадает только в половине случаев.**

**В датасете изначально было 75% строк с тарифом Ультра. Но мы не знаем, как модель разделила выборки и в какую из них Ультра и Смарт в каком соотношении попали.**